In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malayalam-news/HackFake Database for the Hackathon - Sheet1.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [3]:
#Read dataset
FILEPATH='/kaggle/input/malayalam-news'
df_train=pd.read_csv(os.path.join(FILEPATH,'HackFake Database for the Hackathon - Sheet1.csv'))[:500]
df_train = df_train.drop(['Website (source)', 'Timeline/Date','Title'], axis=1)[2:].reset_index(drop=True)
df_train=df_train.dropna(axis=0)

In [4]:
df_train.columns

Index(['Heading of News Article', 'Text of News Article', 'Hate speech',
       'Misleading', 'Disinformation', 'Rumor/Hoax', 'Sensationalism',
       'Credible'],
      dtype='object')

In [5]:
X=df_train[['Text of News Article']]
#'Heading of News Article'
y=df_train[['Hate speech','Misleading','Disinformation','Rumor/Hoax','Sensationalism']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train=np.array(y_train).astype('int')
y_test=np.array(y_test).astype('int')

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X.shape[0]*20,
  decay_rate=1,
  staircase=False)


def tf_model():
    # Input layer
    input_layer = Input(shape=(), dtype=tf.string, name='text_input')

    # Preprocessor layer
    preprocessor_layer = hub.KerasLayer(
        "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
    )
    preprocessed_text = preprocessor_layer(input_layer)

    # Encoder layer
    encoder_layer = hub.KerasLayer(
        "https://tfhub.dev/google/LaBSE/2",
        trainable=False
    )

    embedded_text = encoder_layer(preprocessed_text)["sequence_output"]

    # Pass the embeddings through LSTM layer
    lstm_layer = LSTM(units=64, return_sequences=True)(embedded_text)

    # Bidirectional layer
    bi_dir_layer = Bidirectional(LSTM(64, return_sequences=False))(lstm_layer)

    # Batch normalization
    norm_layer = BatchNormalization()(bi_dir_layer)

    # Dense layer
    den_layer = Dense(units=32, activation='relu')(norm_layer)

    # Dropout layer
    drop_layer = Dropout(0.2)(den_layer)

    # Dense layer
    penum_layer = Dense(units=16, activation='relu')(drop_layer)

    # Output layer
    output_layer = Dense(units=5, activation='sigmoid', name='output')(penum_layer)

    # Create the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

# Create the model
model = tf_model()

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_mask': (None, 128)   0         ['text_input[0][0]']          
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [7]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['acc']
)

In [8]:
model.fit(X_train,y_train,epochs=10,callbacks=[early_stopping],validation_data=[X_test,y_test],batch_size=200)

Epoch 1/10
2/2 [==============================] - 100s 54s/step - loss: 0.6951 - acc: 0.3133 - val_loss: 0.6751 - val_acc: 0.2727
Epoch 2/10
2/2 [==============================] - 85s 51s/step - loss: 0.6217 - acc: 0.3554 - val_loss: 0.6724 - val_acc: 0.2545
Epoch 3/10
2/2 [==============================] - 100s 55s/step - loss: 0.5872 - acc: 0.3886 - val_loss: 0.6702 - val_acc: 0.2303
Epoch 4/10
2/2 [==============================] - 100s 65s/step - loss: 0.5410 - acc: 0.3855 - val_loss: 0.6676 - val_acc: 0.2303
Epoch 5/10
2/2 [==============================] - 89s 54s/step - loss: 0.5131 - acc: 0.3705 - val_loss: 0.6656 - val_acc: 0.2303
Epoch 6/10
2/2 [==============================] - 88s 54s/step - loss: 0.4818 - acc: 0.3434 - val_loss: 0.6649 - val_acc: 0.2242
Epoch 7/10
2/2 [==============================] - 88s 53s/step - loss: 0.4556 - acc: 0.3705 - val_loss: 0.6640 - val_acc: 0.2303
Epoch 8/10
2/2 [==============================] - 88s 53s/step - loss: 0.4278 - acc: 0.3735 - 

In [9]:
model.predict(X_train.iloc[5])

1/1 [==============================] - 2s 2s/step


array([[0.47245365, 0.3859915 , 0.39482236, 0.32087725, 0.43905604]],
      dtype=float32)

In [10]:
y_train[5]

array([1, 0, 0, 0, 0])

In [11]:
model.predict(X_test.iloc[10])

1/1 [==============================] - 0s 233ms/step


array([[0.48835486, 0.46789652, 0.4641324 , 0.4580982 , 0.5362996 ]],
      dtype=float32)

In [12]:
y_test[10]

array([0, 0, 0, 0, 1])

In [13]:
model.predict([''])

1/1 [==============================] - 2s 2s/step


array([[0.48557222, 0.4470237 , 0.39126027, 0.37361187, 0.44724014]],
      dtype=float32)

In [14]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))